In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
import pandas as pd

In [2]:
df = pd.read_csv("training_dataset_dp2.csv")

In [3]:
X = df[['Num_escuelas',
 'Num_hospitales',
 'Num_iglesias',
 'Num_zonas_tipicas',
  "Hora_llegada_1",                                          
  "Hora_llegada_2",
  "Hora_llegada_3",                                          
  "Hora_llegada_4",
  "Hora_llegada_5",                                          
  "Hora_llegada_6",
  "Hora_llegada_7",                                          
  "Hora_llegada_8",
  "Hora_llegada_9",                                          
  "Hora_llegada_10",
  "Hora_llegada_11",                                          
  "Hora_llegada_12",
  "Hora_llegada_13",                                          
  "Hora_llegada_14",
  "Hora_llegada_15",                                          
  "Hora_llegada_16",                                         
  "Hora_llegada_17",                                          
  "Hora_llegada_18",
  "Hora_llegada_19",                                          
  "Hora_llegada_20",
  "Hora_llegada_21",                                          
  "Hora_llegada_22",
  "Hora_llegada_23", ]]

y = df["Num_viajes"]

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42)

In [5]:
linear_model = LinearRegression()
linear_model.fit(X, y)

LinearRegression()

In [6]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_log_error as msle
import numpy as np 

yfit_linear = pd.DataFrame(linear_model.predict(X_train), index = X_train.index, columns = ["Num_viajes"]).clip(0.)

ypred_linear = pd.DataFrame(linear_model.predict(X_test), index = X_test.index, columns = ["Num_viajes"]).clip(0.)


print('Linear RMSLE :', np.sqrt(msle(y_test, ypred_linear)))
print('Linear MAE :', mae(y_test, ypred_linear))


Linear RMSLE : 1.4360590284591315
Linear MAE : 862.5127815297449


# Evaluación del modelo

Para evaluar el modelo, revisaremos el % de error utilizando MAE

### Training error

In [7]:
dataframe = pd.DataFrame()
dataframe['Num_viajes'] = y_train
dataframe['Num_viajes_p'] = yfit_linear
dataframe['dif'] = abs(dataframe['Num_viajes'] - dataframe['Num_viajes_p'])
import math
dataframe['Num_viajes_p'] =  dataframe['Num_viajes_p'].apply(lambda x: math.ceil(x))
dataframe["Dif_perc"] =  dataframe["dif"] / dataframe["Num_viajes"] *100
print("The % of error in training is", dataframe["Dif_perc"].mean())


The % of error in training is 44.04807085947441


### Testing error

In [8]:
dataframe = pd.DataFrame()
dataframe['Num_viajes'] = y_test
dataframe['Num_viajes_p'] = ypred_linear
dataframe['dif'] = abs(dataframe['Num_viajes'] - dataframe['Num_viajes_p'])
import math
dataframe['Num_viajes_p'] =  dataframe['Num_viajes_p'].apply(lambda x: math.ceil(x))
dataframe["Dif_perc"] =  dataframe["dif"] / dataframe["Num_viajes"] *100
print("The % of error in test is", dataframe["Dif_perc"].mean())


The % of error in test is 54.92606358208962


## Interpretación

In [9]:
import statsmodels.api as sm
X = sm.add_constant(X_train)
model = sm.OLS(y_train,X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             Num_viajes   R-squared:                       0.767
Model:                            OLS   Adj. R-squared:                  0.760
Method:                 Least Squares   F-statistic:                     120.8
Date:                Mon, 28 Nov 2022   Prob (F-statistic):          3.45e-291
Time:                        19:04:12   Log-Likelihood:                -8547.5
No. Observations:                1020   AIC:                         1.715e+04
Df Residuals:                     992   BIC:                         1.729e+04
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const             -1837.0358    169.70

Al analizar los resultados de la regresión lineal podemos ver que el impacto que hace el intercepto (la constante) ahora sí es estadísticamente significativo y tiene una alta magnitud. 

Asímismo vemos que el número de escuelas tiene una magnitud pequeña, y se relaciona de manera positiva con el número de viajes recibido. Es decir que a mayor escuelas, mayor cantidad de viajes recibidos en una comuna. 

Con respecto a los hospitales, vemos que es estadísticamente significativo (p value menor a 0.05), y es la variable más significativa de todas al tener el t value más alto. También tiene una magnitud mediana. Esta variable se relaciona positivamente con la cantidad de viajes recibidos en una comuna, es decir que a mayor cantidad de hospitales, mayor cantidad de viajes recibidos. 

Hablando de las iglesias, esta variable es de una magnitud alta y es estadísticamente significativa. Se relaciona positivamente con los viajes recibidos por comuna, por lo que a mayor cantidad de iglesias, más viajes se reciben en una comuna.

En lo que respecta a las zonas típicas, esta variable es de una magnitud alta y estadísticamente significativa. Tiene una relación negativa con la cantidad de viajes, así que mientras más zonas típicas se tengan, menos viajes se recibe en la comuna.

Con respecto a las variables dummies, vemos que todas ellas son estadísticamente significativas para nuestro modelo, además de que tienen magnitudes mucho más altas que el resto de variables, por lo que sabemos que el cambio en la hora tiene un impacto más grande en la cantidad de viajes recibidos que las demás variables. 